In [7]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from sbcbinaryformat import Streamer, Writer
import scipy.signal as sg
from pathlib import Path
import re

ana_path = "../LAr10Ana/"
sys.path.insert(0, ana_path)

from GetEvent import GetEvent, GetRun

%matplotlib inline

In [14]:
recon_path = Path("/exp/e961/data/SBC-25-recon/dev-output/")
data_path = Path("/exp/e961/data/SBC-25-daqdata/")

In [44]:
def get_info(runs, data_path="/exp/e961/data/SBC-25-daqdata/", recon_path="/exp/e961/data/SBC-25-recon/dev-output/"):
    runs = set(runs)
    total_evs = 0
    total_clock_time = 0
    total_ev_time = 0
    total_livetime = 0

    for run in runs:
        livetime = Streamer(os.path.join(recon_path, run, "exposure.sbc")).to_dict()["PT1101_livetime"]
        total_evs += len(livetime)
        total_livetime += np.sum(livetime)/3600

        event_info = Streamer(os.path.join(recon_path, run, "event.sbc")).to_dict()
        total_ev_time += np.sum(event_info["ev_livetime"])/1000/3600

        # start time of first event to end time of last event
        start_time, end_time = event_info["start_time"], event_info["end_time"]
        total_clock_time += (end_time[-1]-start_time[0])/3600

    # total event livetime, number of events, total livetime at pressure
    print(f"Total event time: \t{total_ev_time:.1f}h \nTotal clock time: \t{total_clock_time:.1f}h \nTotal evs: \t\t{total_evs} \nTotal livetime: \t{total_livetime:.1f}h")


In [1]:
def gen_run_numbers(run_ranges):
    # input is a tuple or a list of tuples, which contains (date, start_run, end_run)
    if isinstance(run_ranges, tuple):
        run_ranges = [run_ranges]

    ret = [
        f"{date}_{run}"
        for date, start, end in run_ranges
        for run in range(start, end + 1)
    ]

    return ret

In [16]:
def get_runs_between(start_folder, end_folder, data_path=Path("/exp/e961/data/SBC-25-daqdata/")):
    pattern = re.compile(r'^\d{8}_\d+.*$')
    all_folders = sorted(
        [f.stem for f in data_path.iterdir() if pattern.match(f.name)],
        key=lambda x: (x.split('_')[0], int(x.split('_')[1].split('_')[0] if '_' in x else x.split('_')[1]))
    )
    
    if start_folder not in all_folders:
        raise ValueError(f"Start folder '{start_folder}' not found")
    if end_folder not in all_folders:
        raise ValueError(f"End folder '{end_folder}' not found")
    
    start_idx = all_folders.index(start_folder)
    end_idx = all_folders.index(end_folder)
    
    if start_idx > end_idx:
        start_idx, end_idx = end_idx, start_idx
    
    return all_folders[start_idx:end_idx + 1]

In [53]:
cf_images_runs = ["20260205_5", "20260205_6", "20260205_7", "20260205_8", "20260205_9", "20260205_10", "20260205_11", "20260205_12", 
                  "20260205_13", "20260205_14", "20260205_15", "20260205_16", "20260205_17", "20260205_18", 
                  "20260206_0", "20260206_1", "20260206_2", "20260206_3", "20260206_4", "20260206_5", "20260206_6", "20260206_7"]
get_info(cf_images_runs)

Total event time: 	14.9h 
Total clock time: 	23.7h 
Total evs: 		991 
Total livetime: 	3.7h


In [54]:
cf_scint_images_runs = ["20260212_0", "20260212_1", "20260212_4", "20260212_5", "20260212_6", "20260212_7"]
get_info(cf_scint_images_runs)

Total event time: 	3.0h 
Total clock time: 	9.7h 
Total evs: 		217 
Total livetime: 	0.5h


In [55]:
co_runs = ["20260206_8", "20260206_9", "20260206_10", "20260206_11", "20260206_12"]
get_info(co_runs)

Total event time: 	0.8h 
Total clock time: 	1.4h 
Total evs: 		66 
Total livetime: 	0.1h


In [52]:
bg_runs = ["20260206_21", "20260206_22", "20260206_23", "20260206_24", "20260206_25", 
           "20260207_0", "20260207_1", "20260207_2", "20260207_3", "20260207_4", "20260207_5", 
           "20260208_0", "20260208_1", "20260208_2", "20260208_3", "20260208_4", "20260208_5", 
           "20260208_6", "20260208_7", "20260208_8", "20260208_9", "20260208_10", 
           "20260209_0", "20260209_1", "20260209_2", "20260209_3", "20260209_4", ]
get_info(bg_runs)

Total event time: 	27.4h 
Total clock time: 	49.9h 
Total evs: 		1316 
Total livetime: 	12.2h


In [51]:
bg_scint_runs = ["20260209_11", "20260209_12", "20260209_13"]
get_info(bg_scint_runs)

Total event time: 	1.5h 
Total clock time: 	2.3h 
Total evs: 		82 
Total livetime: 	0.5h


In [50]:
cf252_scint_data = ["20260213_1", "20260213_2", "20260213_3"]
get_info(cf252_scint_data)

Total event time: 	1.7h 
Total clock time: 	3.9h 
Total evs: 		119 
Total livetime: 	0.4h


In [49]:
cf252_data = ["20260213_4", "20260213_5", "20260213_6", "20260213_7", "20260213_8", "20260213_9", 
              "20260214_0", "20260214_1", "20260214_2", "20260214_3", "20260214_4", "20260214_5", "20260214_6", 
              "20260214_7", "20260214_8", "20260214_9", "20260214_10", "20260214_11", "20260214_12", "20260214_13", "20260214_14", 
              "20260215_0", "20260215_1", "20260215_2", "20260215_3", "20260215_4", "20260215_5", "20260215_6", 
              "20260215_7", "20260215_8", "20260215_9", "20260215_10", "20260215_11", "20260215_12", "20260215_13", "20260215_14", 
              "20260216_0", "20260216_1", "20260216_2", "20260216_3", "20260216_4", "20260216_5", "20260216_6", 
              "20260216_7", "20260216_8", "20260216_9", "20260216_10", "20260216_11", "20260216_12", "20260216_13", 
              "20260217_0", "20260217_1", "20260217_2", "20260217_3", "20260217_4", "20260217_5", "20260217_6",]
get_info(cf252_data)

Total event time: 	40.6h 
Total clock time: 	90.3h 
Total evs: 		2805 
Total livetime: 	9.0h


In [48]:
bg_data = ["20260217_7", "20260217_8", "20260217_9", "20260217_10", "20260217_11", "20260217_12", "20260217_13", 
           "20260218_0", "20260218_1", "20260218_2", "20260218_3", "20260218_4", "20260218_5", "20260218_6", 
           "20260218_15", "20260218_16", "20260219_0", "20260219_1", "20260219_2", "20260219_3", "20260219_4", 
           "20260219_5", "20260219_6", "20260219_7", "20260219_8", "20260219_9", "20260219_10", "20260219_11",  
           "20260220_0", ]#"20260220_1", "20260220_2", "20260220_3", "20260220_4"]
get_info(bg_data)

Total event time: 	27.9h 
Total clock time: 	51.9h 
Total evs: 		1367 
Total livetime: 	12.2h


In [47]:
bg_scint_data = ["20260218_9", "20260218_10", "20260218_11", "20260218_12", "20260218_13", "20260218_14"]
get_info(bg_scint_data)

Total event time: 	3.8h 
Total clock time: 	6.8h 
Total evs: 		176 
Total livetime: 	1.7h


In [46]:
bg_data = get_runs_between("20260220_1", "20260220_4")
get_info(bg_data)

Total event time: 	3.5h 
Total clock time: 	6.6h 
Total evs: 		170 
Total livetime: 	1.6h


In [45]:
co_data = get_runs_between("20260220_5", "20260226_8")
get_info(co_data)

Total event time: 	63.4h 
Total clock time: 	143.4h 
Total evs: 		4464 
Total livetime: 	12.2h
